In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/pymc-devs/pymc3/master/pymc3/examples/data/radon.csv')

In [3]:
df.iloc[0]  # print first row of data

Unnamed: 0            0
idnum              5081
state                MN
state2               MN
stfips               27
zip               55735
region                5
typebldg              1
floor                 1
room                  3
basement              N
windoor                
rep                   2
stratum               4
wave                 41
starttm             930
stoptm              930
startdt           12088
stopdt            12288
activity            2.2
pcterr              9.7
adjwt            1146.5
dupflag               1
zipflag               0
cntyfips              1
county           AITKIN
fips              27001
Uppm           0.502054
county_code           0
log_radon      0.832909
Name: 0, dtype: object

In [4]:
# make column for intercept / bias term
df['intercept'] = 1
# select feature columns for X
X_data = df[['intercept', 'adjwt', 'Uppm', 'stfips', 'activity', 'pcterr']].values
# select output column for y
y_data = df[['log_radon']].values


In [5]:
N, k = X_data.shape
N, k

(919, 6)

In [6]:
X_train, X_test = X_data[:600], X_data[600:]
y_train, y_test = y_data[:600], y_data[600:]

In [7]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((600, 6), (319, 6), (600, 1), (319, 1))

In [8]:
with tf.name_scope('input'):
    X = tf.placeholder(tf.float32, [None, k])
    y = tf.placeholder(tf.float32, [None, 1])
with tf.name_scope('training'):
    with tf.name_scope('model'):
        W = tf.Variable(tf.zeros([k, 1]))
        y_hat = tf.matmul(X, W)
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(tf.square(y_hat - y))
        optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
        train_step = optimizer.minimize(loss)


In [9]:
initializer = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(initializer)
    for epoch in range(0, 20000 + 1):
        sess.run(train_step, feed_dict={X:X_train, y:y_train})
        if epoch % 1000 == 0:
            y_pred = sess.run(y_hat, feed_dict={X:X_test})
            r_squared_test = 1 - tf.reduce_mean(tf.square(y_pred - y_test)) / tf.reduce_mean(tf.square(y_test))
            y_pred = sess.run(y_hat, feed_dict={X:X_train})
            r_squared_train = 1 - tf.reduce_mean(tf.square(y_pred - y_train)) / tf.reduce_mean(tf.square(y_train))
            print("Epoch: %06d | Train R^2: %06f | Test R^2: %06f" % (
                epoch,
                sess.run(r_squared_train),
                sess.run(r_squared_test)))

Epoch: 000000 | Train R^2: 0.129958 | Test R^2: 0.092816
Epoch: 001000 | Train R^2: 0.892442 | Test R^2: 0.920049
Epoch: 002000 | Train R^2: 0.911469 | Test R^2: 0.941411
Epoch: 003000 | Train R^2: 0.915276 | Test R^2: 0.946933
Epoch: 004000 | Train R^2: 0.915539 | Test R^2: 0.947718
Epoch: 005000 | Train R^2: 0.915586 | Test R^2: 0.947759
Epoch: 006000 | Train R^2: 0.915614 | Test R^2: 0.947728
Epoch: 007000 | Train R^2: 0.915622 | Test R^2: 0.947685
Epoch: 008000 | Train R^2: 0.915624 | Test R^2: 0.947676
Epoch: 009000 | Train R^2: 0.915624 | Test R^2: 0.947665
Epoch: 010000 | Train R^2: 0.915624 | Test R^2: 0.947671
Epoch: 011000 | Train R^2: 0.915624 | Test R^2: 0.947671
Epoch: 012000 | Train R^2: 0.915624 | Test R^2: 0.947671
Epoch: 013000 | Train R^2: 0.915624 | Test R^2: 0.947671
Epoch: 014000 | Train R^2: 0.915624 | Test R^2: 0.947671
Epoch: 015000 | Train R^2: 0.915624 | Test R^2: 0.947671
Epoch: 016000 | Train R^2: 0.915624 | Test R^2: 0.947671
Epoch: 017000 | Train R^2: 0.91